Load a liabrary and data

In [29]:
import pandas as pd
from textblob import TextBlob
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr


# Load headline data (using your provided dataset)
news_df = pd.read_csv('../data/raw_analyst_ratings.csv', parse_dates=['Date'])
news_df['Date'] = pd.to_datetime(news_df['Date'], format='mixed')

# Load stock data (using your provided dataset)
stock_df = pd.read_csv('../data/yfinance_data/TSLA_historical_data.csv', parse_dates=['Date'])
stock_df['Date'] = stock_df['Date'].dt.date




C:\Users\Administrator\AppData\Local\Temp\ipykernel_19276\2400745963.py:10: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  news_df['Date'] = pd.to_datetime(news_df['Date'], format='mixed')


Sentiment Analysis on Headlines

In [ ]:
# 1. Sentiment Analysis
def analyze_sentiment(text):
    analysis = TextBlob(text)
    return analysis.sentiment.polarity  # Returns polarity score between -1 and 1

news_df['sentiment'] = news_df['headline'].apply(analyze_sentiment)

# Classify sentiment
news_df['sentiment_label'] = pd.cut(news_df['sentiment'],
                                   bins=[-1, -0.1, 0.1, 1],
                                   labels=['negative', 'neutral', 'positive'])

# 2. Calculate Daily Stock Returns
stock_df['daily_return'] = stock_df['Close'].pct_change() * 100  # Percentage returns

# 3. Merge datasets on date
combined_df = pd.merge(stock_df, news_df, on='Date', how='left')

# Forward fill sentiment for days without news (simple approach)
combined_df['sentiment'].fillna(method='ffill', inplace=True)
combined_df['sentiment_label'].fillna(method='ffill', inplace=True)

# 4. Correlation Analysis

# Create a temporary DataFrame with only the relevant columns and drop NaNs from both simultaneously
temp_df = combined_df[['sentiment', 'daily_return']].dropna()

# Now perform the correlation on the cleaned data
correlation, p_value = pearsonr(temp_df['sentiment'], temp_df['daily_return'])

print(f"Pearson Correlation: {correlation:.3f}")
print(f"P-value: {p_value:.3f}")

# 5. Visualization
plt.figure(figsize=(14, 10))

# Plot 1: Sentiment and Returns Over Time
plt.subplot(2, 2, 1)
plt.plot(combined_df['Date'], combined_df['sentiment'], label='Sentiment Score', color='blue')
plt.plot(combined_df['Date'], combined_df['daily_return'], label='Daily Returns (%)', color='green', alpha=0.7)
plt.title('Sentiment vs. Stock Returns Over Time')
plt.ylabel('Score/Returns')
plt.legend()
plt.grid(True)

# Plot 2: Sentiment Distribution
plt.subplot(2, 2, 2)
sns.countplot(x='sentiment_label', data=combined_df, order=['negative', 'neutral', 'positive'])
plt.title('Distribution of News Sentiment')
plt.xlabel('Sentiment Category')
plt.ylabel('Count')

# Plot 3: Returns by Sentiment Category
plt.subplot(2, 2, 3)
sns.boxplot(x='sentiment_label', y='daily_return', data=combined_df, 
            order=['negative', 'neutral', 'positive'])
plt.title('Daily Returns by Sentiment Category')
plt.xlabel('Sentiment Category')
plt.ylabel('Daily Return (%)')

# Plot 4: Correlation Scatter Plot
plt.subplot(2, 2, 4)
sns.regplot(x='sentiment', y='daily_return', data=combined_df,
            scatter_kws={'alpha':0.5}, line_kws={'color':'red'})
plt.title(f'Sentiment-Return Correlation (r = {correlation:.2f})')
plt.xlabel('Sentiment Score')
plt.ylabel('Daily Return (%)')

plt.tight_layout()
plt.show()

C:\Users\Administrator\AppData\Local\Temp\ipykernel_19276\191771364.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  combined_df['sentiment'].fillna(method='ffill', inplace=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_19276\191771364.py:20: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df['sentiment'].fillna(method='ffill', inplace=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_19276\191771364.py:21: Fu

ValueError: `x` and `y` must have the same length along `axis`.